# Installing OWASP Juice Shop on Kubernetes

This notebook provides a simple and concise guide to install OWASP Juice Shop in a separate Kubernetes namespace.

## Prerequisites
- Kubernetes cluster running (minikube, kind, or cloud provider)
- kubectl configured and connected to your cluster
- Basic understanding of Kubernetes concepts


## Step 1: Create a New Namespace

First, we'll create a dedicated namespace for Juice Shop to keep it isolated from other applications.


In [ ]:
# Create the namespace   
kubectl create namespace juice-shop

# Verify the namespace was created
kubectl get namespaces


In [ ]:
## Step 2: Create Deployment YAML

Create a deployment configuration for Juice Shop. This will define how the application runs in the cluster.


In [ ]:
# Create deployment YAML file
cat << EOF > juice-shop-deployment.yaml
apiVersion: apps/v1
kind: Deployment
metadata:
  name: juice-shop
  namespace: juice-shop
  labels:
    app: juice-shop
spec:
  replicas: 1
  selector:
    matchLabels:
      app: juice-shop
  template:
    metadata:
      labels:
        app: juice-shop
    spec:
      containers:
      - name: juice-shop
        image: bkimminich/juice-shop:latest
        ports:
        - containerPort: 3000
        resources:
          requests:
            memory: "256Mi"
            cpu: "250m"
          limits:
            memory: "512Mi"
            cpu: "500m"
EOF


In [ ]:
## Step 3: Create Service YAML

Create a service to expose the Juice Shop application within the cluster and potentially externally.


In [ ]:
# Create service YAML file
cat << EOF > juice-shop-service.yaml
apiVersion: v1
kind: Service
metadata:
  name: juice-shop-service
  namespace: juice-shop
  labels:
    app: juice-shop
spec:
  type: NodePort
  ports:
  - port: 3000
    targetPort: 3000
    nodePort: 30000
  selector:
    app: juice-shop
EOF


In [ ]:
## Step 4: Deploy to Kubernetes

Apply the configuration files to deploy Juice Shop to your Kubernetes cluster.

### Method 1: Individual YAML Files
Apply each file separately for granular control.


In [ ]:
# Apply the deployment
kubectl apply -f juice-shop-deployment.yaml

# Apply the service
kubectl apply -f juice-shop-service.yaml

# Check the deployment status
kubectl get deployments -n juice-shop

# Check the pods
kubectl get pods -n juice-shop

# Check the service
kubectl get services -n juice-shop


In [ ]:
### Method 2: Using Kustomize (Recommended)

Kustomize is a built-in Kubernetes tool that allows you to manage multiple YAML files as a single unit. This is more efficient and easier to manage.


In [ ]:
# First, create a kustomization.yaml file
cat << EOF > kustomization.yaml
apiVersion: kustomize.config.k8s.io/v1beta1
kind: Kustomization

metadata:
  name: juice-shop
  namespace: juice-shop

resources:
- juice-shop-deployment.yaml
- juice-shop-service.yaml

commonLabels:
  app: juice-shop
  component: web-application
EOF


In [ ]:
# Deploy everything with a single command using Kustomize
kubectl apply -k .

# Check the deployment status
kubectl get all -n juice-shop

# Verify that common labels were applied
kubectl get pods -n juice-shop --show-labels


In [ ]:
### Kustomize Benefits

- **Single Command**: Deploy all resources with `kubectl apply -k .`
- **Easy Management**: Update all resources together


In [ ]:
## Step 5: Access the Application

Once deployed, you can access Juice Shop through different methods depending on your cluster setup.


In [ ]:
# For Minikube users - get the service URL
minikube service juice-shop-service -n juice-shop --url

# For other clusters - port forward to access locally
kubectl port-forward service/juice-shop-service 3000:3000 -n juice-shop




In [ ]:
## Step 6: Verify Installation

Check that everything is running correctly and troubleshoot if needed.


In [ ]:
# Check pod logs if there are issues
kubectl logs -l app=juice-shop -n juice-shop

# Describe the deployment for detailed information
kubectl describe deployment juice-shop -n juice-shop

# Check all resources in the namespace
kubectl get all -n juice-shop


In [ ]:
## Cleanup (Optional)

When you're done testing, you can easily remove everything by deleting the namespace.


In [ ]:
# Option 1: Delete using Kustomize (if you used kustomize to deploy)
kubectl delete -k .

# Option 2: Delete the entire namespace (this removes all resources)
kubectl delete namespace juice-shop

# Clean up the YAML files
rm juice-shop-deployment.yaml juice-shop-service.yaml kustomization.yaml
